In [85]:
import numpy as np
import pandas as pd

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
iris = fetch_ucirepo(id=53) 
  
# data (as pandas dataframes) 
X = iris.data.features 
y = iris.data.targets 
  
# metadata 
# print(iris.metadata)

# variable information
# print(iris.variables)

In [116]:
from sklearn.model_selection import KFold
from sklearn.neighbors import KDTree

X = np.array(iris.data.features)
y = np.array(iris.data.targets)

n_splits = 10

k = int(input("Enter how many neighbours to be considered: "))

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

accuracy = np.empty(0)

for fold, (train_index, test_index) in enumerate(kf.split(X), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #print(f"Fold {fold}")
    #print(f"Train data shape: {X_train.shape}, Test data shape: {X_test.shape}")
    #print("-" * 40)

    guessed = 0

    # Build the KDTree
    kdtree = KDTree(X_train)

    for index, sample in enumerate(X_test):
        distances = np.linalg.norm(X_train - sample, axis=1)
    
        indices = np.argsort(distances)
        distances, indices = kdtree.query([sample], k)
        closestK = X_train[indices][0]
        closestKLabels = y_train[indices][0]
        #closestK = X_train[indices[:k]]
        #closestKLabels = y_train[indices[:k]]

        def distToSampleScore(x):
            return 1/(np.linalg.norm(sample - x)+0.001);

        vectorizedDist = np.vectorize(distToSampleScore)

        scores = np.apply_along_axis(distToSampleScore, axis=1, arr=closestK)

        types = np.unique(closestKLabels)        

        sum_scores = np.array([np.sum(scores[np.array(np.where(closestKLabels == label))[0]]) for label in types])

        res = types[np.argmax(sum_scores)]
    
        #print(f"Test sample: {sample}")
        #print(f"Closest {k} training samples:\n{closest_k}")
        #types, counts = np.unique(closestKLabels, return_counts=True)
        #res_index = np.argmax(counts)
        #res = types[res_index]
        #frequency = counts[res_index]
        #print(f"Res label: {res}")
        guessed += (res == y_test[index])

        #if(res != y_test[index]):
        #    print("wrong")
        #    print(types)
        #    print(sum_scores)
        #    print(y_test[index])
        #    print(res)
        #    print("end wrong")
        #print("-" * 40)

    currentSampleAccuracy = guessed / X_test.shape[0]
    #print(f"Guessed: {currentSampleAccuracy}")
    accuracy = np.append(accuracy, currentSampleAccuracy)
    #print(accuracy)

Enter how many neighbours to be considered:  10


In [117]:
print(accuracy)
print(np.mean(accuracy))

[1.         1.         1.         0.93333333 1.         0.93333333
 0.93333333 1.         1.         0.93333333]
0.9733333333333334
